In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
import math
from scipy.stats import norm

## Taux de complétion

Hypothèse : Comparer le taux de complétion entre le groupe Test (nouveau design) et le groupe Contrôle (ancien design).

Objectif : Vérifier si la différence est statistiquement significative.

Actions : Définir les hypothèses nulle et alternative, tester avec le niveau de signification approprié, analyser le p-value et d'autres mesures statistiques.

Hypothèse nulle (H0) : Il y a une différence significative dans le taux de complétion entre le groupe Test (nouveau design) et le groupe Contrôle (ancien design).

Hypothèse alternative (H1) : Il n'y a pas de différence significative dans le taux de complétion entre le groupe Test (nouveau design) et le groupe Contrôle (ancien design).

In [13]:
alpha = 0.05

control_completion_rate = 0.36  # 36%
test_completion_rate = 0.45  # 45%
control_sample_size = 15428  
test_sample_size = 18682  

control_completions = int(control_completion_rate * control_sample_size)
test_completions = int(test_completion_rate * test_sample_size)
print(f"control : {control_completions}, test : {test_completions}")

control_failures = control_sample_size - control_completions
test_failures = test_sample_size - test_completions


control_data = [1] * control_completions + [0] * control_failures
test_data = [1] * test_completions + [0] * test_failures


t_stat, p_value_ttest = st.ttest_ind(control_data, test_data, equal_var=False)

t_stat, p_value_ttest

print(f"result stats : {t_stat: .2f}, result p_val : {p_value_ttest: .2f} ")

if p_value_ttest < alpha:
    print("H0 rejected : The completion rates between the Control and Test groups are different")
else:
    print("H1 rejected : There is no difference in the completion rates between the Control and Test groups")

control : 5554, test : 8406
result stats : -16.94, result p_val :  0.00 
H0 rejected : The completion rates between the Control and Test groups are different


## Taux de complétion avec seuil de rentabilité

Hypothèse : Le nouveau design doit augmenter le taux de complétion d'au moins 5% pour être rentable.

Objectif : Confirmer que l'amélioration observée atteint ou dépasse ce seuil.

Actions : Effectuer une analyse pour vérifier si l'augmentation est suffisamment importante pour justifier les coûts.

Hypothèse nulle (H0) : L'augmentation du taux de complétion du groupe Test (nouveau design) est inférieure à 5% par rapport au groupe Contrôle (ancien design).

Hypothèse alternative (H1) : L'augmentation du taux de complétion du groupe Test (nouveau design) est égale ou supérieure à 5% par rapport au groupe Contrôle (ancien design).

In [16]:
threshold_increase = 0.05  # 5% improvement
observed_increase = test_completion_rate - control_completion_rate


se_diff = math.sqrt(
    (control_completion_rate * (1 - control_completion_rate)) / control_sample_size +
    (test_completion_rate * (1 - test_completion_rate)) / test_sample_size
)
z_stat_threshold = (observed_increase - threshold_increase) / se_diff

p_value_threshold = 1 - norm.cdf(z_stat_threshold)

z_stat_threshold, p_value_threshold

if p_value_threshold > threshold_increase:
    print("The improvement in the Test group's completion rate is less than 5%.")
else:
    print(f"The improvement in the Test group's completion rate is 5% or more.")


The improvement in the Test group's completion rate is 5% or more.


## Autres exemples d'hypothèses

Comparer l'âge moyen des clients utilisant le nouveau et l'ancien processus.

Comparer la durée d'ancienneté des clients utilisant le nouveau et l'ancien processus.

Analyser les différences de genre dans l'engagement avec le nouveau ou l'ancien processus.

Hypothèse nulle (H0) : L'âge moyen des clients utilisant le nouveau processus est le même que celui des clients utilisant l'ancien processus.

Hypothèse alternative (H1) : L'âge moyen des clients utilisant le nouveau processus est différent de celui des clients utilisant l'ancien processus.

Hypothèse nulle (H0) : L'ancienneté moyenne des clients utilisant le nouveau processus est la même que celle des clients utilisant l'ancien processus.

Hypothèse alternative (H1) : L'ancienneté moyenne des clients utilisant le nouveau processus est différente de celle des clients utilisant l'ancien processus.

Hypothèse nulle (H0) : Il n'y a pas de différence dans l'engagement avec le nouveau processus entre les genres.

Hypothèse alternative (H1) : Il y a une différence dans l'engagement avec le nouveau processus entre les genres.